# Examples of Assistory Library: Basics

Table of content
- Item
- Resource node
- Building
- Recipe
- Schematic

See [Class Dependencies](../library_programming_guide.md) for more details

In [1]:
import sys, os
os.chdir('..')

assert 'assistory' in os.listdir(os.getcwd())
sys.path.append(os.getcwd())

from assistory import game

## Item
Item data is provided as key value pairs with a dictionary of attributes as value

In [2]:
item_name = list(game.ITEMS)[0]
print('ITEMS Count:', len(game.ITEMS), '. Example:', item_name)
game.ITEMS[item_name]

ITEMS Count: 180 . Example: BP_EqDescZipLine_C


{'slug': 'zipline',
 'className': 'BP_EqDescZipLine_C',
 'name': 'Zipline',
 'sinkPoints': 5284,
 'description': 'Provides faster traversal of factories by allowing pioneers to zip along Power lines.\nActivate the Zipline and aim at a nearby Power Line to connect to it.',
 'stackSize': 1,
 'energyValue': 0,
 'radioactiveDecay': 0,
 'liquid': False,
 'fluidColor': {'r': 255, 'g': 255, 'b': 255, 'a': 0}}

Items have solid, liquid or gas form. Gas and liquid behave equally. Hence, only `ITEM_NAMES_LIQUID` is defined

In [3]:
item_name = game.ITEM_NAMES_LIQUID[0]
print('ITEM_NAMES_LIQUID Count:', len(game.ITEM_NAMES_LIQUID), '. Example:', item_name)
game.ITEMS[item_name]

ITEM_NAMES_LIQUID Count: 15 . Example: Desc_AluminaSolution_C


{'slug': 'alumina-solution',
 'className': 'Desc_AluminaSolution_C',
 'name': 'Alumina Solution',
 'sinkPoints': 20,
 'description': 'Dissolved Alumina, extracted from Bauxite. Can be further refined into Aluminum Scrap for Aluminum Ingot production.',
 'stackSize': 50000,
 'energyValue': 0,
 'radioactiveDecay': 0,
 'liquid': True,
 'fluidColor': {'r': 193, 'g': 193, 'b': 193, 'a': 0}}

Similarly, items with special characteristics are summarized in `ITEM_NAMES_RADIOACTIVE`, `ITEM_NAMES_NON_SELLABLE` and `ITEM_NAMES_EXTRACTION`.

To represent an amount of multiple different items, use the class `ItemValues`. Compared to a simple `dict` it checks validity of the item names and always represents to amount of all items in `ITEMS`.

In [4]:
items = game.ItemValues(
    {
        'Desc_AluminaSolution_C': 100, # liters
        'BP_EqDescZipLine_C': 5.5 # pieces
    }
)
print(items)

{'Desc_AluminaSolution_C': 100, 'BP_EqDescZipLine_C': 5.5}


In [5]:
try:
    game.ItemValues({'Desc_RandomItemName_C': 1}) # will raise an error
except ValueError as e:
    print(e)

Unknown key: Desc_RandomItemName_C. Did you mean: "Desc_Computer_C"?


Note:
- The unit of liquid (and gas) items is liters and the unit of solid items is pieces
- Differently than in the game, the amounts are floating point values (Required for optimization)

Items can be loaded and stored

In [6]:
items2 = game.ItemValues.load('example/example_data/base_item_rate.yml')
print(items2.round(3))
items2.save('/tmp/items_out.yml')

{'Desc_Cable_C': 30.0, 'Desc_Cement_C': 20.0, 'Desc_IronPlateReinforced_C': 10.5, 'Desc_Rotor_C': 10.0, 'Desc_SpaceElevatorPart_2_C': 10.0, 'Desc_Stator_C': 1.0, 'Desc_SteelPipe_C': 8.25, 'Desc_Wire_C': 22.0}


Classes that are subclass `Values`, like `ItemValues` support addition and subtraction.

In [7]:
print('I - I = ', items - items)
print('I + I = ', items + items)
print(sum(items2.values()))
items2 -= items
print(sum(items2.values()))

I - I =  {}
I + I =  {'Desc_AluminaSolution_C': 200, 'BP_EqDescZipLine_C': 11.0}
111.75
6.25


To represent a set of different items, use the class `ItemFlags` subclassing `set`. Compared to a simple `set` it checks validity of the item names and `ITEMS`

In [8]:
item_flags = game.ItemFlags({'BP_EqDescZipLine_C'})
try:
    item_flags.add('Desc_FooBar_C')
except ValueError as e:
    print(e)

Unknown flag: "Desc_FooBar_C". Did you mean: "Desc_Rotor_C"?


Classes that are a subclass of `Flags`, like `ItemFlags` support ``as_array``. It returns a numpy array with a boolean value for each possible element, e.g. each item in `ITEMS`, whether the element in contained in the ``Flags`` object.

In [9]:
flag_array = item_flags.as_array()
for item_name, contained, _ in zip(sorted(game.ITEMS), flag_array, range(5)):
    print(item_name, 'is', 'contained' if contained else 'not contained')
print('...')

BP_EqDescZipLine_C is contained
BP_EquipmentDescriptorGasmask_C is not contained
BP_EquipmentDescriptorHazmatSuit_C is not contained
BP_EquipmentDescriptorHoverPack_C is not contained
BP_EquipmentDescriptorJetPack_C is not contained
...


## Resource Node
`RESOURCE_NODES` defines resource node types analog to `ITEMS`

In [10]:
node_name = list(game.RESOURCE_NODES)[0]
print('RESOURCE_NODES Count:', len(game.RESOURCE_NODES), '. Example:', node_name)
game.RESOURCE_NODES[node_name]

RESOURCE_NODES Count: 15 . Example: Desc_Stone_C-non_fracking


{'resource_name': 'Desc_Stone_C',
 'method_name': 'non_fracking',
 'extraction_recipes': ('Recipe_MinerMk1Stone_C',
  'Recipe_MinerMk2Stone_C',
  'Recipe_MinerMk3Stone_C')}

Besides, the real node names in the game can be used to lookup the type of the node using `UNIQUE_NODES`. `NODE_RECIPES_AVAILABLE` summarizes the amount of each node type.

In [11]:
unique_node_name = list(game.UNIQUE_NODES)[0]
print('UNIQUE_NODES Count:', len(game.UNIQUE_NODES), '. Example:', unique_node_name)
print(game.UNIQUE_NODES[unique_node_name])

print('Available recipe amount of this node type:', game.NODE_RECIPES_AVAILABLE[game.get_resource_node_name('Desc_OreIron_C', fracking=False)])

UNIQUE_NODES Count: 608 . Example: BP_ResourceNode462_UAID_40B076DF2F7907E201_1624182051
{'resource': 'Desc_OreIron_C', 'fracking': False, 'purity': 'impure'}
Available recipe amount of this node type: 153.5


## Building
`BUILDINGS` defines factory buildings analog to ``ITEMS``.

In [12]:
building_name = list(game.BUILDINGS)[0]
print('BUILDINGS Count:', len(game.BUILDINGS), '. Example:', building_name)

game.BUILDINGS[building_name]

BUILDINGS Count: 23 . Example: Desc_AlienPowerBuilding_C


{'power_consumption': 0,
 'costs': {'Desc_WAT1_C': 10, 'Desc_SAMFluctuator_C': 50, 'Desc_Cable_C': 100, 'Desc_SteelPlateReinforced_C': 50, 'Desc_Motor_C': 25, 'Desc_Computer_C': 10}}

`BuildingValues` provide additional methods over the ones from `Values`

In [13]:
buildings = game.BuildingValues(
    {
        'Desc_AlienPowerBuilding_C': 1,
        'Desc_AssemblerMk1_C': 2
    }
)
print('Costs:')
buildings.get_costs().pprint(0)
print('Power balance:', buildings.get_power_balance())

Costs:
Desc_Cable_C                120
Desc_Computer_C             10
Desc_IronPlateReinforced_C  16
Desc_Motor_C                25
Desc_Rotor_C                8
Desc_SAMFluctuator_C        50
Desc_SteelPlateReinforced_C 50
Desc_WAT1_C                 10
Power balance: -30.0


## Recipe
`RECIPES` defines factory buildings analog to ``ITEMS``.

In [14]:
recipe_name = 'Recipe_AILimiter_C'
print('RECIPES Count:', len(game.RECIPES), '. Example:', recipe_name)
game.RECIPES[recipe_name]

RECIPES Count: 371 . Example: Recipe_AILimiter_C


{'name': 'AI Limiter',
 'ingredients': {'Desc_CopperSheet_C': 5, 'Desc_HighSpeedWire_C': 20},
 'products': {'Desc_CircuitBoardHighSpeed_C': 1},
 'producedIn': {'Desc_AssemblerMk1_C'},
 'time': 12.0,
 'manualTimeMultiplier': 1.0}

`RecipeValues` provide special methods

In [15]:
recipes = game.RecipeValues({
    'Recipe_MinerMk2OreCopper_C': 50 / 120,
    'Recipe_IngotCopper_C': 50 / 30,
    'Recipe_CopperSheet_C': 25 / 10,
    'Recipe_AILimiter_C': 1,
})
print('Item rate per minute:')
recipes.get_item_rate_balance().round(3).pprint(0)
print('\nRequried buidings:')
recipes.get_buildings().round(3).pprint(0)
print('\nResource nodes used:')
recipes.get_resource_nodes_used().round(3).pprint(0)

Item rate per minute:
Desc_CircuitBoardHighSpeed_C 5.0
Desc_HighSpeedWire_C         -100.0

Requried buidings:
Desc_AssemblerMk1_C   1
Desc_ConstructorMk1_C 2.5
Desc_MinerMk2_C       0.417
Desc_SmelterMk1_C     1.667

Resource nodes used:
Desc_OreCopper_C-non_fracking 0.417


A subset of recipes can (only or optionally) be executed by hand, i.e. without automation, which implies a different speed. These recipes are named in `RECIPE_NAMES_HANDCRAFTED`.

In [16]:
recipes2 = game.RecipeValues(
    {'Recipe_AILimiter_C': 1},
    omega=game.RECIPE_NAMES_HANDCRAFTED
)
print('Item rate per minute:')
recipes2.get_item_rate_balance_handcraft().pprint(0)

Item rate per minute:
Desc_CircuitBoardHighSpeed_C 40.0
Desc_CopperSheet_C           -200.0
Desc_HighSpeedWire_C         -800.0


`RecipeFlags` allows summaries of involved items and buildings

In [17]:
recipe_flags = game.RecipeFlags({'Recipe_MinerMk1Stone_C', 'Recipe_Concrete_C'})
print('Items consumed:', sorted(recipe_flags.get_items_consumed()))
print('Items produced:', sorted(recipe_flags.get_items_produced()))
print('Buildings involved:', sorted(recipe_flags.get_buildings_involved()))

Items consumed: ['Desc_Stone_C']
Items produced: ['Desc_Cement_C', 'Desc_Stone_C']
Buildings involved: ['Desc_ConstructorMk1_C', 'Desc_MinerMk1_C']


# Schematic
`SCHEMATICS` defines schematics analog to `ITEMS`. They include research at the MAM, milestones and project assembly phases

In [18]:
schematic_name = 'Schematic_3-4_C'
print('SCHEMATICS Count:', len(game.SCHEMATICS), '. Example:', schematic_name)
game.SCHEMATICS[schematic_name]

SCHEMATICS Count: 557 . Example: Schematic_3-4_C


{'name': 'Basic Steel Production',
 'costs': {'Desc_ModularFrame_C': 50, 'Desc_Rotor_C': 150, 'Desc_Cement_C': 500, 'Desc_Wire_C': 1000},
 'unlock_recipes': {'Recipe_IngotSteel_C',
  'Recipe_SpaceElevatorPart_2_C',
  'Recipe_SteelBeam_C',
  'Recipe_SteelPipe_C'},
 'unlock_buildings': {'Desc_FoundryMk1_C'}}

Unlocked schematics make available recipes and building. The building in which a recipe is used needs to be unlocked to include the recipe in the `get_recipes_unlocked` function.

In [19]:
schematic_flags = game.SchematicFlags([
    'Schematic_3-4_C',
    'Schematic_Alternate_SteelPipe_Molded_C',
])
all_recipes_unlocked = (
    game.SCHEMATICS['Schematic_3-4_C']['unlock_recipes']
    | game.SCHEMATICS['Schematic_Alternate_SteelPipe_Molded_C']['unlock_recipes']
)
print('Recipes potentially unlocked:', sorted(all_recipes_unlocked))
print('Buildings required:', sorted(all_recipes_unlocked.get_buildings_involved()))
print('Buildings unlocked:', sorted(schematic_flags.get_buildings_unlocked()))
print('Recipes unlocked:', sorted(schematic_flags.get_recipes_unlocked()))


Recipes potentially unlocked: ['Recipe_Alternate_SteelPipe_Molded_C', 'Recipe_IngotSteel_C', 'Recipe_SpaceElevatorPart_2_C', 'Recipe_SteelBeam_C', 'Recipe_SteelPipe_C']
Buildings required: ['Desc_AssemblerMk1_C', 'Desc_ConstructorMk1_C', 'Desc_FoundryMk1_C']
Buildings unlocked: ['Desc_FoundryMk1_C']
Recipes unlocked: ['Recipe_Alternate_SteelPipe_Molded_C', 'Recipe_IngotSteel_C']
